### The goal of this assignment is to implement your own logistic regression classifier. You will:

#### Extract features from Amazon product reviews.
#### Convert an SFrame into a NumPy array.
#### Implement the link function for logistic regression.
#### Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
#### Implement gradient ascent.
#### Given a set of coefficients, predict sentiments.
#### Compute classification accuracy for the logistic regression model.

#### For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

#### Note: For this assignment, we eliminated class imbalance by choosing a subset of the data with a similar number of positive and negative reviews.
#### After that, try counting the number of positive and negative reviews.

In [2]:
import numpy as np
import pandas as pd
import string
from math import sqrt

In [3]:
products = pd.read_csv('amazon_baby_subset.csv')

In [4]:
products.head()

,name,review,rating,sentiment
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1


#### Apply text cleaning on the review data

#### In this section, we will perform some simple feature cleaning using data frames. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into the JSON file named important_words.json. Load the words into a list important_words.

#### Let us perform 2 simple data transformations:
##### Remove punctuation
##### Compute word counts (only for important_words)

In [5]:
important_words = pd.read_json('important_words.json')

In [6]:
# before fill nan
np.sum(pd.isna(products['review']))

241

In [7]:
# fill in N/A's in the review column
products = products.fillna({'review': ''})

In [8]:
# after fill nan
np.sum(pd.isna(products['review']))

0

In [59]:
len(products[products['sentiment'] == 1])

26579

In [60]:
len(products[products['sentiment'] == -1])

26493

#### Write a function remove_punctuation that takes a line of text and removes all punctuation from that text. The function should be analogous to the following Python code:

In [9]:
def remove_punctuation(text):
    # https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python
    table = str.maketrans({key: None for key in string.punctuation})
    return text.translate(table)

#### Apply the remove_punctuation function on every element of the review column and assign the result to the new column review_clean. Note. Many data frame packages support apply operation for this type of task. Consult appropriate manuals.

In [10]:
products['review_clean'] = products['review'].apply(remove_punctuation)

In [11]:
products.head()

,name,review,rating,sentiment,review_clean
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...


#### Now we proceed with the second item. For each word in important_words, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in important_words which keeps a count of the number of times the respective word occurs in the review text.

In [12]:
important_words_list = important_words[0].tolist()

In [13]:
for i in important_words_list:
    products[i] = products['review_clean'].apply(lambda x: x.split().count(i))

#### After #4 and #5, the data frame products should contain one column for each of the 193 important_words. As an example, the column perfect contains a count of the number of times the word perfect occurs in each of the reviews.
#### Now, write some code to compute the number of product reviews that contain the word perfect.
#### How many reviews contain the word perfect?

In [14]:
len(products[products['perfect'] >= 1])

2955

### Convert data frame to multi-dimensional array
#### It is now time to convert our data frame to a multi-dimensional array. Look for a package that provides a highly optimized matrix operations. In the case of Python, NumPy is a good choice.
#### Write a function that extracts columns from a data frame and converts them into a multi-dimensional array. We plan to use them throughout the course, so make sure to get this function right.
#### The function should accept three parameters:

##### dataframe: a data frame to be converted
##### features: a list of string, containing the names of the columns that are used as features.
##### label: a string, containing the name of the single column that is used as class labels.
#### The function should return two values
##### one 2D array for features
##### one 1D array for class labels
#### The function should do the following:
##### Prepend a new column constant to dataframe and fill it with 1's. This column takes account of the intercept term. Make sure that the constant column appears first in the data frame.
##### Prepend a string 'constant' to the list features. Make sure the string 'constant' appears first in the list.
##### Extract columns in dataframe whose names appear in the list features.
##### Convert the extracted columns into a 2D array using a function in the data frame library. If you are using Pandas, you would use as_matrix() function.
##### Extract the single column in dataframe whose name corresponds to the string label.
##### Convert the column into a 1D array.
##### Return the 2D array and the 1D array.

In [15]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    # use as_matrix() function 
    feature_matrix = dataframe.as_matrix(columns=features)
    label_array = dataframe.as_matrix(columns=[label])
    return feature_matrix, label_array

#### Using the function written in #8, extract two arrays feature_matrix and sentiment. The 2D array feature_matrix would contain the content of the columns given by the list important_words. The 1D array sentiment would contain the content of the column sentiment.
#### Quiz Question: How many features are there in the feature_matrix?
#### Quiz Question: Assuming that the intercept is present, how does the number of features in feature_matrix relate to the number of features in the logistic regression model?

In [69]:
feature_matrix, label_array = get_numpy_data(products, important_words_list, 'sentiment')

In [70]:
label_array.shape

(53072, 1)

### Estimating conditional probability with link function
#### P(yi=+1|xi,w)=11+exp(−wTh(xi))
#### where the feature vector h(x_i) represents the word counts of important_words in the review x_i. Write a function named predict_probability that implements the link function.
#### Take two parameters: feature_matrix and coefficients. First compute the dot product of feature_matrix and coefficients. Then compute the link function P(y = +1 | x,w). Return the predictions given by the link function. Your code should be analogous to the following Python function:

In [18]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    # score is len(feature_matrix) * 1  vector
    score = np.dot(feature_matrix, coefficients)
    
    # Compute P(y_i = +1 | x_i, w) using the link function
    # YOUR CODE HERE
    # predictions is len(feature_matrix) * 1  vector
    predictions = 1 / (1 + np.exp(-score))
    
    # return predictions
    return predictions

In [82]:
predict_probability(feature_matrix, np.zeros(194)).shape

(53072,)

### Compute derivative of log likelihood with respect to a single coefficient
#### We will now write a function feature_derivative that computes the derivative of log likelihood with respect to a single coefficient w_j. The function accepts two arguments:
##### errors: vector whose i-th value contains 1（y_i = +1) - P(y_i = +1| x_i, w)
##### feature: vector whose i-th value contains h j(x i)
#### This corresponds to the j-th column of feature_matrix.

#### The function should do the following:

##### Take two parameters errors and feature.
##### Compute the dot product of errors and feature.
##### Return the dot product. This is the derivative with respect to a single coefficient w_j.

In [20]:
def feature_derivative(errors, feature):     
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
        # Return the derivative
    return derivative

#### In the main lecture, our focus was on the likelihood. In the advanced optional video, however, we introduced a transformation of this likelihood---called the log-likelihood---that simplifies the derivation of the gradient and is more numerically stable. Due to its numerical stability, we will use the log-likelihood instead of the likelihood to assess the algorithm.

The log-likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):

#### Write a function compute_log_likelihood that implements the equation. The function would be analogous to the following Python function:



In [172]:
def compute_log_likelihood(feature_matrix, sentiment, coefficients):
    
    # indicator is a vector 
    indicator = (sentiment == 1).reshape(len(sentiment),)
    
    # socres is also a vector, each element is the score of each obervation 
    scores = np.dot(feature_matrix, coefficients)
    
    # lp is scalar of the total dataset
    lp = np.sum((indicator - 1) * scores - np.log(1. + np.exp(-scores)))
    
    return lp 

### There is a huge difference in below:

In [114]:
np.array([1, -1]) - 1

array([ 0, -2])

In [115]:
np.array([True, False]) - 1

array([ 0, -1])

### Taking gradient steps
#### Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum.Write a function logistic_regression to fit a logistic regression model using gradient ascent.The function accepts the following parameters:
##### feature_matrix: 2D array of features
##### sentiment: 1D array of class labels
##### initial_coefficients: 1D array containing initial values of coefficients
##### step_size: a parameter controlling the size of the gradient steps
##### max_iter: number of iterations to run gradient ascent
#### The function returns the last set of coefficients after performing gradient ascent.
#### The function carries out the following steps:
##### Initialize vector coefficients to initial_coefficients.
##### Predict the class probability P(y_i = +1| x_i, w) using your predict_probability function and save it to variable predictions.
##### Compute indicator value for (y_i = +1） by comparing sentiment against +1. Save it to variable indicator.
##### Compute the errors as difference between indicator and predictions. Save the errors to variable errors.
##### For each j-th coefficient, compute the per-coefficient derivative by calling feature_derivative with the j-th column of feature_matrix. Then increment the j-th coefficient by (step_size*derivative).
##### Once in a while, insert code to print out the log likelihood.
##### Repeat steps 2-6 for max_iter times.

In [107]:
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = initial_coefficients # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        predictions = predict_probability(feature_matrix, coefficients)

        # Compute indicator value for (y_i = +1) (sentiment == 1).reshape(len(sentiment),)
        indicator = (sentiment == 1).reshape(len(sentiment),)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(len(coefficients)): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:, j])

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size * derivative

        # Checking whether log likelihood is increasing
        
        if itr <= 15 or (itr <= 100 and itr % 10 == 0) or (itr <= 1000 and itr % 100 == 0) \
        or (itr <= 10000 and itr % 1000 == 0) or itr % 10000 == 0:
            lp = compute_log_likelihood(feature_matrix, sentiment, coefficients)
            print('iteration %*d: log likelihood of observed labels = %.8f' % \
                (int(np.ceil(np.log10(max_iter))), itr, lp))
    return coefficients

#### Quiz question: As each iteration of gradient ascent passes, does the log likelihood increase or decrease?

In [108]:
initial_coefficients = np.zeros(194)
step_size = 1e-7
max_iter = 301

In [171]:
np.zeros(194).shape

(194,)

In [109]:
my_cofficients = logistic_regression(feature_matrix, label_array, np.zeros(194), 1e-7, 301)

iteration   0: log likelihood of observed labels = -36780.91768478
iteration   1: log likelihood of observed labels = -36775.13434712
iteration   2: log likelihood of observed labels = -36769.35713564
iteration   3: log likelihood of observed labels = -36763.58603240
iteration   4: log likelihood of observed labels = -36757.82101962
iteration   5: log likelihood of observed labels = -36752.06207964
iteration   6: log likelihood of observed labels = -36746.30919497
iteration   7: log likelihood of observed labels = -36740.56234821
iteration   8: log likelihood of observed labels = -36734.82152213
iteration   9: log likelihood of observed labels = -36729.08669961
iteration  10: log likelihood of observed labels = -36723.35786366
iteration  11: log likelihood of observed labels = -36717.63499744
iteration  12: log likelihood of observed labels = -36711.91808422
iteration  13: log likelihood of observed labels = -36706.20710739
iteration  14: log likelihood of observed labels = -36700.5020

In [110]:
my_cofficients.shape

(194,)

### Now, we write some code to compute class predictions. We do this in two steps:

#### First compute the scores using feature_matrix and coefficients using a dot product.
#### Then apply threshold 0 on the scores to compute the class predictions. Refer to the formula above.

In [111]:
scores_new = np.dot(feature_matrix, my_cofficients)

In [118]:
sum(scores_new > 0)

25126

### Measuring accuracy
#### accuracy = # correctly classified data points / # total data points
#### What is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)

In [143]:
new_label = np.array([1 if item > 0 else -1 for item in scores_new])

In [155]:
np.sum(label_array.reshape(len(label_array), ) == new_label) / len(new_label)

0.7518653904130238

### Which words contribute most to positive & negative sentiments
#### Recall that in the earlier assignment, we were able to compute the "most positive words". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:

##### Treat each coefficient as a tuple, i.e. (word, coefficient_value). The intercept has no corresponding word, so throw it out.
##### Sort all the (word, coefficient_value) tuples by coefficient_value in descending order. Save the sorted list of tuples to word_coefficient_tuples.

#### Ten "most positive" words

##### 18. Compute the 10 words that have the most positive coefficient values. These words are associated with positive sentiment. Quiz question: Which word is not present in the top 10 "most positive" words?
##### 19. Next, we repeat this exerciese on the 10 most negative words. That is, we compute the 10 words that have the most negative coefficient values. These words are associated with negative sentiment.

In [163]:
useful_coef_df = pd.DataFrame(my_cofficients[1:], columns=['Coef Value'])

In [166]:
useful_coef_df['Word'] = important_words_list

In [170]:
useful_coef_df.sort_values('Coef Value', ascending=False).head(10)

,Coef Value,Word
2,0.066546,great
3,0.065891,love
7,0.064795,easy
8,0.045436,little
22,0.044976,loves
11,0.030135,well
33,0.029740,perfect
10,0.020078,old
34,0.018409,nice
20,0.017703,daughter


In [169]:
useful_coef_df.sort_values('Coef Value').head(10)

,Coef Value,Word
5,-0.053860,would
18,-0.041511,product
96,-0.038982,money
77,-0.033070,work
32,-0.030051,even
105,-0.028979,disappointed
12,-0.028712,get
28,-0.027743,back
113,-0.026593,return
97,-0.024482,monitor
